In [13]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import statsmodels.api as sm

# Загрузка данных из файла Excel
file_path = "C:\\Users\\nsedoff\\Documents\\VSCode docs\\ITiABD-PM22-7-Nikolay-Sedov\\econometrics\\multikol.xlsx"

data = pd.read_excel(file_path)

# Отображение первых нескольких строк данных для ознакомления
data.head()


""


In [7]:
# Проверка всех листов в файле Excel
sheets = pd.ExcelFile(file_path)
sheet_names = sheets.sheet_names
sheet_names


['Лист1', 'Лист2']

In [8]:
# Загрузка данных с каждого листа
data_sheet1 = pd.read_excel(file_path, sheet_name=sheet_names[0])
data_sheet2 = pd.read_excel(file_path, sheet_name=sheet_names[1])

data_sheet1.head(), data_sheet2.head()  # Отображение первых строк каждого листа для оценки содержания данных


(Empty DataFrame
 Columns: []
 Index: [],
       Y     X1     X2     X3
 0  15.3  135.2  109.5  125.2
 1  18.6   86.5  111.4  112.2
 2  19.4  107.4  105.3  105.1
 3  18.1  108.1  105.1  112.4
 4  19.9  104.6  112.0  106.2)

In [12]:

# Данные для анализа мультиколлинеарности
data_analysis = data_sheet2[['X1', 'X2', 'X3']]

# Добавление константы для VIF
data_analysis_const = add_constant(data_analysis)

# Вычисление VIF для каждой переменной
vif_data = pd.DataFrame()
vif_data["Feature"] = data_analysis_const.columns
vif_data["VIF"] = [variance_inflation_factor(data_analysis_const.values, i) for i in range(data_analysis_const.shape[1])]

vif_data


,Feature,VIF
0,const,1163.073118
1,X1,1.137075
2,X2,1.147613
3,X3,1.035222


In [14]:

# Метод Феррара-Глобера
farrar_glauber_results = {}

# Функция для вычисления регрессии и проверки значимости коэффициентов
def farrar_glauber_test(dependent_var, independent_vars):
    model = sm.OLS(dependent_var, independent_vars).fit()
    return model.pvalues

# Проведение теста для каждой переменной
for column in data_analysis.columns:
    # Выбор независимых переменных, исключая текущую зависимую переменную
    independent_vars = data_analysis.drop(column, axis=1)
    # Добавление константы к независимым переменным
    independent_vars_const = add_constant(independent_vars)
    # Выполнение теста
    farrar_glauber_results[column] = farrar_glauber_test(data_analysis[column], independent_vars_const)

# Вывод результатов
farrar_glauber_results_df = pd.DataFrame(farrar_glauber_results)
farrar_glauber_results_df


,X1,X2,X3
X1,NaN,1.846485e-01,0.607498
X2,0.184649,NaN,0.528287
X3,0.607498,5.282868e-01,NaN
const,0.042222,9.096106e-10,0.913242
